In [1]:
import numpy as np
import random as rnd

In [2]:
import igl
import scipy as sp
import meshplot as mp
import numpy as np
from meshplot import plot, subplot, interact
import math

In [26]:
v, f = igl.read_triangle_mesh("3dPotteryDs/3D Models/Lagynos/qpvase5.obj")

In [27]:
f.shape, type(f), v.shape, type(v)

((1960, 3), numpy.ndarray, (1024, 3), numpy.ndarray)

In [28]:
def correctDirection(nVector):
    nVector = nVector * np.sign(nVector[np.argmax(abs(nVector))])
    return nVector

In [195]:
TH_RATIO_CRV = 0.9
TH_COSINE = 0.75
#TH_MN_CURVATURE = 0.025

In [30]:
#Compute facet normals
n_faces = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
#Compute vertex normals
n_vertex = igl.per_vertex_normals(v, f)
#v1, v2: principal directions | k1, k2: principal values
v1, v2, k1, k2 = igl.principal_curvature(v, f)

In [31]:
length_x = max(v[:][0]) - min(v[:][0])
length_y = max(v[:][1]) - min(v[:][1])
length_z = max(v[:][2]) - min(v[:][2])

area = 2 * (length_x * length_y + length_x * length_z + length_y * length_z)
diagonal_bbox = math.sqrt(length_x * length_x + length_y * length_y + length_z * length_z)

In [32]:
v_size = min(round(area * 125), round(v.shape[0] / 2))

In [56]:
v_size, round(v.shape[0] / v_size)

(512, 2)

In [267]:
np.percentile(abs(k1), 15), np.percentile(abs(k1), 40), np.percentile(abs(k1), 65), np.percentile(abs(k1), 90)

(0.10252667281446885,
 0.3194790599674767,
 0.5525401716109815,
 1.7620566621015445)

In [268]:
np.percentile(abs(k2), 15), np.percentile(abs(k2), 40), np.percentile(abs(k2), 65), np.percentile(abs(k2), 90)

(0.26357892002127953,
 0.5451515355199719,
 1.6161543821360465,
 5.097082818947928)

In [57]:
v_ratio = round(v.shape[0] / v_size)
pk1 = np.percentile(abs(k1), 10)
pk2 = np.percentile(abs(k2), 10)
v_indices = [] #np.array([0 for i in range(v_size)])

for i in range(0, v_size):
    index = rnd.randint(v_ratio * i, min(v_ratio * (i + 1) - 1, v.shape[0] - 1))
    if abs(k1[index]) > pk1 or abs(k2[index]) > pk2:
        v_indices.append(index) #v_indices[i] = index;

In [59]:
s_size = min(round(area * 25), round(len(v_indices)/2))
s_indices = np.array([0 for i in range(s_size)])
s_size, v_size, s_indices.shape, len(v_indices), v_ratio, v.shape[0]

(254, 512, (254,), 509, 2, 1024)

In [60]:
s_ratio = round(len(v_indices) / s_size)

for i in range(0, s_size):
    index = rnd.randint(s_ratio * i, min(s_ratio * (i + 1) - 1, len(v_indices) - 1));
    s_indices[i] = v_indices[index];

In [482]:
# iPoints : indexes list of points Sampled
def baseReflection(iPoints): 
    sample_size = len(iPoints)
    lxPairs = []
    
    for i in range(0, sample_size - 1):
        for j in range(i + 1, sample_size):
            ix = iPoints[i]
            jx = iPoints[j]
            
            ratio_1 = abs(k1[ix] / k1[jx])
            ratio_2 = abs(k2[ix] / k2[jx])
            
            if TH_RATIO_CRV < ratio_1 < 1/TH_RATIO_CRV and TH_RATIO_CRV < ratio_2 < 1/TH_RATIO_CRV:
                reflexNormal = v[ix] - v[jx]
                reflexNormal = reflexNormal / np.linalg.norm(reflexNormal)
                    
                normalBalance = n_vertex[ix] + n_vertex[jx]
                normalBalance = normalBalance / np.linalg.norm(normalBalance)
                                
                if abs(np.dot(normalBalance, reflexNormal)) < 1 - TH_COSINE:
                    if reflexNormal[np.argmax(abs(reflexNormal))] < 0: #Correct direction
                        reflexNormal = -reflexNormal
                    
                    middlePoint = (v[ix] + v[jx]) / 2                    
                    reflexNormal_D = - np.dot(reflexNormal, middlePoint)
                    origin = np.array([0, 0, 0])
                    projection = projectPointPlane(origin, reflexNormal, reflexNormal_D) 
                    
                    #Use projection point instead of D-component
                    xcPair = [ix, jx, reflexNormal, projection]
                    lxPairs.append(xcPair)
                   
    return lxPairs

In [273]:
lxcPairs = baseReflection(s_indices)
len(lxcPairs)

512

In [404]:
lxcPairs[22]

[259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])]

In [483]:
for i in range(0, len(lxcPairs)):
    distanceMetric = distanceMetricRx(lxcPairs[22], lxcPairs[i])
    if distanceMetric[0] < 0.01 and distanceMetric[1] < 0.25: #relative distance, angle
        print(i, distanceMetric)

22 (0.0, 0.0)
27 (1.0577484483319858e-07, 0.09817492626985724)
78 (2.3141119828297346e-08, 0.09817454387963194)
185 (0.002453352188755538, 0.19634849685190398)
186 (0.003413188580357813, 0.09817467710707292)
227 (0.0024530076321578613, 0.19634908590380687)
233 (0.0028038893131595703, 0.09816014393018387)
241 (0.003672185893630966, 0.19635707823462706)
249 (0.0024530670812220505, 0.19634968064141764)
250 (0.003121760987498162, 0.0)
254 (0.0036715374832441507, 0.19635198782794855)
265 (1.481113423227101e-07, 0.19635231157050412)
269 (3.795926127280127e-07, 2.7739383084482916e-06)
271 (0.0, 0.0)
278 (0.005041819926094839, 0.09817225399172687)
281 (0.006143674411385388, 0.09817806913206695)
295 (0.005041376926852472, 0.09816941695128027)
302 (0.0050405830447189616, 0.09815470335736799)
363 (0.004415535600215444, 0.19635226916952062)
364 (0.005620062736515237, 7.818779996154839e-06)
368 (0.006608250494281115, 0.19635008725626182)
371 (0.005041861329267942, 0.09817150176618136)
374 (0.006607

In [201]:
len(lxcPairs), type(lxcPairs[1]), type(lxcPairs[1][2]), type(lxcPairs[1][2][2])

(512, list, numpy.ndarray, numpy.float64)

In [203]:
ax = 134
bx = 183
v[ax] - v[bx], n_vertex[ax] + n_vertex[bx], np.dot(v[ax] - v[bx], n_vertex[ax] + n_vertex[bx]), k1[ax], k2[ax], k1[bx], k2[bx]

(array([ 0.73575, -0.01815, -0.12484]),
 array([-0.25846642, -1.36020535, -1.40388536]),
 0.00978210572280025,
 0.5158379675871669,
 5.419074537774634,
 0.5185426111136144,
 5.508625957435771)

In [67]:
def projectPointPlane(point, plane, planeD):
    k = point.dot(plane) + planeD
    return point - k * plane

In [561]:
def distanceMetricRx(rxPairA, rxPairB):
    module = np.linalg.norm(rxPairA[2]) * np.linalg.norm(rxPairA[2])
    angle = math.acos(min(1.0, abs(np.dot(rxPairA[2], rxPairB[2])) / module))
    
    origin = np.array([0, 0, 0])
    #projectionA = projectPointPlane(origin, rxPairA[2], rxPairA[3])
    #projectionB = projectPointPlane(origin, rxPairB[2], rxPairB[3])
    distance = np.linalg.norm(rxPairA[3] - rxPairB[3])
    
    return distance / diagonal_bbox, angle

In [184]:
diagonal_bbox

9.314047731018992

In [258]:
def epanechnikov_weight(distance, kernel_bandwidth):
    c_distance = distance[0] * distance[1]
    c_kernel_bandwidth = kernel_bandwidth[0] * kernel_bandwidth[1]
    return max(0.0, 1.0 - (c_distance / c_kernel_bandwidth)) 
    #c_kernel_bandwidth **2

In [522]:
def shiftPair_Rx(rxPair, rxList, kernel_bandwidth):
    totalW = 0.0
    shifted_pair = [rxPair[0], rxPair[1], np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 0.0])]
    
    for i in range(len(rxList)):
        distanceMetric = distanceMetricRx(rxPair, rxList[i])
        
        if distanceMetric[0] < kernel_bandwidth[0] and distanceMetric[1] < kernel_bandwidth[1]:           
            # A list cannot directly handle a mathematical operations, while array can
            weight = epanechnikov_weight(distanceMetric, kernel_bandwidth)
            shifted_pair[2] += rxList[i][2] * weight
            shifted_pair[3] += rxList[i][3] * weight
            totalW += weight
    
    if totalW > 0:
        shifted_pair[2] = shifted_pair[2] / np.linalg.norm(shifted_pair[2])
        shifted_pair[3] = shifted_pair[3] / totalW
        
        return shifted_pair
    else:
        return rxPair

In [278]:
EPSILON = 0.05

#distance / bounding_box, angle
KERNEL_BW = [0.05, 0.05]

In [231]:
KERNEL_BW[0], KERNEL_BW[1]

(0.001, 0.1)

In [289]:
#lxcPairs
def meanShift_Rx(lxPairs, kernel_bandwidth):
    stop_moving = [False for i in range(len(lxPairs))]
    shiftedPairs = lxPairs.copy()
        
    while True:
        max_shift_distance = 0
        
        for i in range(len(lxPairs)):
            if not stop_moving[i]:
                #print('x = ', i, shiftedPairs[i])
                pair_new = shiftPair_Rx(shiftedPairs[i], lxPairs, kernel_bandwidth)  
                
                #print('y = ', i, pair_new)
                distance_metric = distanceMetricRx(pair_new, shiftedPairs[i])
                shift_distance = distance_metric[0] * distance_metric[1]
                max_shift_distance = max(max_shift_distance, shift_distance)
                shiftedPairs[i] = pair_new
                
                if shift_distance < EPSILON:
                    stop_moving[i] = True
                
        if max_shift_distance < EPSILON:
            break
            
    return shiftedPairs

In [292]:
shiftedPairsFx = meanShift_Rx(lxcPairs, KERNEL_BW)

[259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] 22 [259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] (0.0, 0.0)
[259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] 250 [550, 570, array([ 0.70710678,  0.70710678, -0.        ]), array([-0.02056, -0.02056,  0.     ])] (0.003121760987498162, 0.0)
[259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] 269 [584, 600, array([ 0.70710874,  0.70710482, -0.        ]), array([2.50001387e-06, 2.50000000e-06, 0.00000000e+00])] (3.795926127280127e-07, 2.7739383084482916e-06)
[259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] 271 [587, 597, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] (0.0, 0.0)
[259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] 364 [773, 795, array([ 0.70710125,  0.70711231, -0.        ]), array([-0.03701359, -0.03701417,  0.        ])] (0

In [312]:
len(shiftedPairsFx)

512

In [486]:
#distance / bounding_box, angle
CLUSTER_EP = [0.1, 0.1]

#distances_line / #distance_projections
CLUSTER_RT = [0.05, 0.05]

In [371]:
def clusterRx(pairs, shiftedPairs):
    cluster_set = []
    
    for i in range(0, len(shiftedPairs)):
        found = False
        c = 0
        
        for c in range(0, len(cluster_set)):
            distance = distanceMetricRx(shiftedPairs[i], cluster_set[c][0])
            if distance[0] < CLUSTER_EP[0] and distance[1] < CLUSTER_EP[1]:
                found = True
                break
        
        if not found:
            #mode, original points, shifted points
            cluster = [shiftedPairs[i], [], [], []]
            cluster_set.append(cluster)
            c = len(cluster_set) - 1
        
        cluster_set[c][1].append(i)
        cluster_set[c][2].append(pairs[i])
        cluster_set[c][3].append(shiftedPairs[i])
    
    return cluster_set

In [391]:
n_cluster = clusterRx(lxcPairs, shiftedPairsFx)
#n_cluster

In [392]:
len(n_cluster)

128

In [528]:
n_cluster[9]

[[127,
  159,
  array([ 0.83811743, -0.32647235,  0.43700684]),
  array([ 1.88460393, -0.73419254,  0.98287473])],
 [11, 54, 58, 68, 71, 123, 154],
 [[127,
   159,
   array([ 0.8375182 , -0.32672394,  0.43796659]),
   array([ 2.01971781, -0.78791142,  1.05617875])],
  [291,
   363,
   array([ 0.87575126, -0.31365869,  0.36698496]),
   array([ 1.54508682, -0.55338762,  0.6474711 ])],
  [291,
   395,
   array([ 0.86620311, -0.29372365,  0.40425065]),
   array([ 1.67066883, -0.56651257,  0.7796889 ])],
  [292,
   363,
   array([ 0.84831528, -0.34970207,  0.39757974]),
   array([ 1.6214583 , -0.66841579,  0.75992852])],
  [292,
   395,
   array([ 0.83873628, -0.32621157,  0.43601315]),
   array([ 1.74479847, -0.67860837,  0.90702537])],
  [313,
   401,
   array([ 0.82535571, -0.3938982 ,  0.40451471]),
   array([ 1.60413969, -0.7655702 ,  0.78620417])],
  [342,
   371,
   array([ 0.83053951, -0.37785531,  0.40918149]),
   array([ 1.57229828, -0.71531968,  0.77462343])]],
 [[127,
   159,
  

In [419]:
for x in range(0, len(n_cluster)):
    if len(n_cluster[x][1]) > 3:
        print(x, len(n_cluster[x][1]))

9 7
10 13
12 23
14 30
15 12
16 34
17 25
18 27
19 4
20 24
25 26
27 5
28 4
29 26
30 10
32 15
42 21
96 27
103 9
104 6


In [398]:
def clusterRx2(pairs):
    cluster_set = []
    
    for i in range(0, len(pairs)):
        found = False
        c = 0
        
        for c in range(0, len(cluster_set)):
            distance = distanceMetricRx(pairs[i], cluster_set[c][0])
            if distance[0] < CLUSTER_EP[0] and distance[1] < CLUSTER_EP[1]:
                found = True
                break
        
        if not found:
            #mode, original points
            cluster = [pairs[i], [], []]
            cluster_set.append(cluster)
            c = len(cluster_set) - 1
        
        cluster_set[c][1].append(i)
        cluster_set[c][2].append(pairs[i])
    
    return cluster_set

In [399]:
n_cluster2 = clusterRx2(lxcPairs)

In [418]:
for x in range(0, len(n_cluster2)):
    if (22 in n_cluster2[x][1]):
        print(x)

18


In [420]:
for x in range(0, len(n_cluster2)):
    if len(n_cluster2[x][1]) > 3:
        print(x, len(n_cluster2[x][1]))

9 7
10 12
12 23
14 30
15 13
16 34
17 24
18 27
20 24
25 26
27 4
28 4
29 26
30 10
32 20
43 21
78 4
93 28
101 9


In [427]:
def normalized(X):
    return X / np.linalg.norm(X)

In [428]:
A = [1/3, 1/3, 1/3, -3]
B = [1, 0, 0, -2]

U = normalized(np.cross(A[:-1], B[:-1]))
M = np.array((A[:-1], B[:-1], U))
X = np.array((-A[-1], -B[-1], 0.))

In [432]:
U, M, X, np.linalg.solve(M, X)

(array([ 0.        ,  0.70710678, -0.70710678]),
 array([[ 0.33333333,  0.33333333,  0.33333333],
        [ 1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.70710678, -0.70710678]]),
 array([3., 2., 0.]),
 array([2. , 3.5, 3.5]))

In [456]:
def projectPointLine(point, line, pointLine):
    ap = point - pointLine
    ab = line
    return pointLine + np.dot(ap, ab)/np.dot(ab, ab) * ab

In [477]:
def distancePointLine(point, line, pointLine):
    return np.linalg.norm(point - projectPointLine(point, line, pointLine))

In [479]:
distancePointLine(np.array([0, 0, 0]), np.array([0.31361367, 0.27279294, 0.90952212]), np.array([1, 2, 3]))

1.0620432616075952

In [460]:
projectPointLine(np.array([0, 0, 0]), np.array([0.31361367, 0.27279294, 0.90952212]), np.array([1, 2, 3]))

array([-0.12517242,  1.0212828 , -0.26315243])

In [461]:
np.dot([-0.12517242,  1.0212828 , -0.26315243], [0.31361367, 0.27279294, 0.90952212])

-4.523009744517026e-10

In [435]:
lxcPairs[1][2], lxcPairs[1][3], lxcPairs[2][2], lxcPairs[2][3]

(array([0.31361367, 0.27279294, 0.90952212]),
 array([1.56130636, 1.35808284, 4.52799987]),
 array([0.10147684, 0.0345258 , 0.99423861]),
 array([0.49051945, 0.16689105, 4.8059574 ]))

In [439]:
normalized(np.cross(lxcPairs[1][2], lxcPairs[2][2]))

array([ 0.73665951, -0.67427918, -0.05177213])

In [466]:
-np.dot(lxcPairs[1][2], lxcPairs[1][3]), -np.dot(lxcPairs[2][2], lxcPairs[2][3])

(-4.978438444982922, -4.833806832055721)

In [480]:
def planesIntersection(pairM, pairN):
    U = normalized(np.cross(pairM[2], pairN[2]))
    M = np.array((pairM[2], pairN[2], U))
    X = np.array((np.dot(pairM[2], pairM[3]), np.dot(pairN[2], pairN[3]), 0.))
    
    origin = np.array([0, 0, 0])
    return correctDirection(U), projectPointLine(origin, U, np.linalg.solve(M, X))

In [481]:
L = planesIntersection(lxcPairs[1], lxcPairs[2])
L

(array([ 0.73665951, -0.67427918, -0.05177213]),
 array([1.32836568, 1.09128052, 4.6883425 ]))

In [467]:
pM = lxcPairs[1]
pN = lxcPairs[2]
np.dot(L[1],pM[2]) - np.dot(pM[2],pM[3]), np.dot(L[1],pN[2]) - np.dot(pN[2],pN[3])

(0.0, -8.881784197001252e-16)

In [470]:
k1[lxcPairs[1][0]], k2[lxcPairs[1][0]]

(0.08947648543259511, 5.275059058765836)

In [493]:
not_parallel = np.dot(lxcPairs[2][2], lxcPairs[3][2]) < TH_RATIO_CRV
not_parallel, np.dot(lxcPairs[2][2], lxcPairs[3][2]), lxcPairs[2][2], lxcPairs[3][2]

(False,
 0.9986857980835615,
 array([0.10147684, 0.0345258 , 0.99423861]),
 array([ 0.13308172, -0.00571861,  0.99108857]))

In [505]:
def BaseContinuousRotation(pairs):
    lrPairs = []
    for m in range(0, len(pairs) - 1):
        for n in range(m+1, len(pairs)):
            rpM = pairs[m]
            rpN = pairs[n]            
            ratio_k1 = max(k1[rpM[0]], k1[rpM[1]]) / max(k1[rpN[0]], k1[rpN[0]])
            ratio_k2 = max(k2[rpM[0]], k2[rpM[1]]) / max(k2[rpN[0]], k2[rpN[0]])
            not_parallel = np.dot(rpM[2], rpN[2]) < TH_RATIO_CRV
            
            if TH_RATIO_CRV < ratio_k1 < 1 / TH_RATIO_CRV and TH_RATIO_CRV < ratio_k2 < 1 / TH_RATIO_CRV and not_parallel:
                #print(m, rpM[2], n, rpN[2])
                line = planesIntersection(rpM, rpN)
                
                p1_i = v[rpM[0]]
                p2_i = v[rpN[0]]                
                distance_1i = distancePointLine(p1_i, line[0], line[1])
                projection_1i = projectPointLine(p1_i, line[0], line[1])
                
                distance_2i = distancePointLine(p2_i, line[0], line[1])
                projection_2i = projectPointLine(p2_i, line[0], line[1])
                
                if np.linalg.norm(projection_1i - projection_2i) < CLUSTER_EP[0] and distance_1i - distance_2i < CLUSTER_EP[0]:
                    #line[0], line[1]
                    rtPair = [rpM[0], rpM[1], rpN[0], rpN[1], line]
                    lrPairs.append(rtPair)
    return lrPairs

In [506]:
lcrPairs = BaseContinuousRotation(lxcPairs)

In [507]:
len(lcrPairs)

5057

In [540]:
0.5*lcrPairs[4][4][0], 0.5*lcrPairs[4][4][1]

(array([-0. ,  0. ,  0.5]),
 array([3.03754315e-06, 1.68672731e-06, 0.00000000e+00]))

In [545]:
shifted_pair = [lcrPairs[4][0], lcrPairs[4][1], lcrPairs[4][2], lcrPairs[4][3], [np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 0.0])]]

In [546]:
shifted_pair

[259, 260, 259, 273, [array([0., 0., 0.]), array([0., 0., 0.])]]

In [518]:
va = np.array([-0.,  0.,  1.])
vb = np.array([-0.,  0.,  1.])
vc = np.dot(va, vb)
math.acos(vc)

0.0

In [562]:
def distanceMetricRt(rtPairA, rtPairB):
    module = np.linalg.norm(rtPairA[4][0]) * np.linalg.norm(rtPairB[4][0])
    angle = math.acos(min(1.0, abs(np.dot(rtPairA[4][0], rtPairB[4][0])) / module))
        
    origin = np.array([0, 0, 0])
    distance = np.linalg.norm(rtPairA[4][1] - rtPairB[4][1])
    
    return distance / diagonal_bbox, angle

In [548]:
def shiftPair_Rt(rtPair, rtList, kernel_bandwidth):
    totalW = 0.0
    shifted_pair = [rtPair[0], rtPair[1], rtPair[2], rtPair[3], [np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 0.0])]]
    
    for i in range(len(rtList)):
        distanceMetric = distanceMetricRt(rtPair, rtList[i])
        
        if distanceMetric[0] < kernel_bandwidth[0] and distanceMetric[1] < kernel_bandwidth[1]:            
            # A list cannot directly handle a mathematical operations, while array can
            weight = epanechnikov_weight(distanceMetric, kernel_bandwidth)
            shifted_pair[4][0] += rtList[i][4][0] * weight
            shifted_pair[4][1] += rtList[i][4][1] * weight
            totalW += weight
    
    if totalW > 0:
        shifted_pair[4][0] = shifted_pair[4][0] / np.linalg.norm(shifted_pair[4])
        shifted_pair[4][1] = shifted_pair[4][1] / totalW
        
        return shifted_pair
    else:
        return rtPair

In [554]:
#lxcPairs
def meanShift_Rt(lrPairs, kernel_bandwidth):
    stop_moving = [False for i in range(len(lrPairs))]
    shiftedPairs = lrPairs.copy()
        
    while True:
        max_shift_distance = 0
        
        for i in range(len(lrPairs)):
            if not stop_moving[i]:
                pair_new = shiftPair_Rt(shiftedPairs[i], lrPairs, kernel_bandwidth)  
                
                distance_metric = distanceMetricRt(pair_new, shiftedPairs[i])
                shift_distance = distance_metric[0] * distance_metric[1]
                max_shift_distance = max(max_shift_distance, shift_distance)
                shiftedPairs[i] = pair_new
                
                if shift_distance < EPSILON:
                    stop_moving[i] = True
                
        if max_shift_distance < EPSILON:
            break
            
    return shiftedPairs

In [550]:
def clusterRt(pairs, shiftedPairs):
    cluster_set = []
    
    for i in range(0, len(shiftedPairs)):
        found = False
        c = 0
        
        for c in range(0, len(cluster_set)):
            distance = distanceMetricRt(shiftedPairs[i], cluster_set[c][0])
            if distance[0] < CLUSTER_EP[0] and distance[1] < CLUSTER_EP[1]:
                found = True
                break
        
        if not found:
            #mode, original points, shifted points
            cluster = [shiftedPairs[i], [], [], []]
            cluster_set.append(cluster)
            c = len(cluster_set) - 1
        
        cluster_set[c][1].append(i)
        cluster_set[c][2].append(pairs[i])
        cluster_set[c][3].append(shiftedPairs[i])
    
    return cluster_set

In [563]:
shiftedPairsRt = meanShift_Rt(lcrPairs, KERNEL_BW)

KeyboardInterrupt: 

In [ ]:
#Usar Kd-Tree o prescindir del mean shift.

In [531]:
n_clusterRt = clusterRt(lrPairs, shiftedPairsRt)

In [532]:
len(n_clusterRt)

128

In [564]:
def clusterRt2(pairs):
    cluster_set = []
    
    for i in range(0, len(pairs)):
        found = False
        c = 0
        
        for c in range(0, len(cluster_set)):
            distance = distanceMetricRt(pairs[i], cluster_set[c][0])
            if distance[0] < CLUSTER_EP[0] and distance[1] < CLUSTER_EP[1]:
                found = True
                break
        
        if not found:
            #mode, original points
            cluster = [pairs[i], [], []]
            cluster_set.append(cluster)
            c = len(cluster_set) - 1
        
        cluster_set[c][1].append(i)
        cluster_set[c][2].append(pairs[i])
    
    return cluster_set

In [567]:
n_cluster2_Rt = clusterRt2(lcrPairs)

In [571]:
for x in range(0, len(n_cluster2_Rt)):
    if len(n_cluster2_Rt[x][1]) > 20:
        print(x, len(n_cluster2_Rt[x][1]))

1 42
4 2015
7 34
8 95
9 67
10 68
11 86
13 60
16 98
18 30
22 23
25 29
28 54
30 46
31 27
32 100
33 37
34 59
38 121
39 23
40 100
42 116
45 28
52 33
54 40
62 41
87 28
92 21
95 30
106 68
139 55
202 47
210 21
231 34


In [574]:
n_cluster2_Rt[4][0], len(n_cluster2_Rt[4][1])

([259,
  260,
  259,
  273,
  (array([-0.,  0.,  1.]),
   array([6.07508630e-06, 3.37345461e-06, 0.00000000e+00]))],
 2015)

In [348]:
distanceMetricRx(shiftedPairsFx[6], n_cluster[0][0])

(0.41745188130705396, 1.2311834058802362)

In [142]:
#x1 = [127, 159, np.array([ 0.8375182 , -0.32672394,  0.43796659]), -2.4115509526708188]
#x2 = [127, 159, np.array([ 0.8375182 , -0.32672394,  0.43796659]), -2.4115509526708188]

x1 = np.array([ 0.8375182 , -0.32672394,  0.43796659])
x2 = np.array([ 0.8375182 , -0.32672394,  0.43796659])
#math.acos(abs(x1[2].dot(x2[2])))
abs(x1.dot(x2)) #math.acos(abs(x1.dot(x2)))

1.0000000022565916

In [130]:
#def DetectionReflectional(baseRx):
shifted_point = [lxcPairs[2][0],  lxcPairs[2][1], np.array([0.0, 0.0, 0.0]), 0]
lista = lxcPairs[2]
shifted_point[2] = lista[2]
shifted_point[3] = lista[3]
shifted_point

In [266]:
distanceMetricRx(lxcPairs[22], lxcPairs[250])[0], distanceMetricRx(lxcPairs[271], lxcPairs[501])[1]

(0.003121760987498162, 0.0)

In [270]:
lDist = []

for i in range(len(lxcPairs)):
    for j in range(i + 1, len(lxcPairs)):
        #lDist.append(distanceMetricRx(lxcPairs[i], lxcPairs[j]))
        origin = np.array([0, 0, 0])
        projectionA = projectPointPlane(origin, lxcPairs[i][2], lxcPairs[i][3])
        projectionB = projectPointPlane(origin, lxcPairs[j][2], lxcPairs[j][3])
        distance = np.linalg.norm(projectionA - projectionB) / diagonal_bbox
        if abs(np.dot(lxcPairs[i][2], lxcPairs[j][2])) > 0.99 and distance < 0.05: #0.98
            print(i, j, distance, math.acos(min(abs(np.dot(lxcPairs[i][2], lxcPairs[j][2])), 1.0)))
        
#np.percentile(lDist, 10), np.percentile(lDist, 25), np.percentile(lDist, 40), np.percentile(lDist, 55), np.percentile(lDist, 70), np.percentile(lDist, 85)

5 9 0.031647149200060565 0.08065183937396787
5 96 0.041042022432112156 0.12421340538182515
8 13 0.029859609030950095 0.08002195502195973
11 71 0.03557233199887842 0.002358426310007194
12 21 0.022999364067637276 0.06752480465123684
12 26 0.022999294455005623 0.10647742269564654
12 73 0.02299928287096976 0.10647678882300755
12 195 0.007376549265121697 0.10352602003586331
12 200 0.0038916022039730443 0.1352021356875273
12 210 0.024685510828101595 0.06752533439716285
12 218 0.02507100186524888 0.10647503031491083
12 229 0.02468455465162073 0.0675255780133651
12 244 0.024269254003008937 0.13049381854284298
12 247 0.025070236871045173 0.1064782972199377
12 252 0.024270341995566318 0.13050248338946888
12 264 0.02299928306605813 0.10647924072549908
12 268 0.02299925074759681 0.10647640429527366
12 270 0.022999097629535066 0.1304992814648881
12 275 0.02300032953735908 0.06752503520427666
12 288 0.026033800077581857 0.06752708150357115
12 294 0.026034167550515432 0.06752427194906221
12 296 0.026

20 423 0.0015506702637420491 0.09817040164551913
20 429 1.1861613167885076e-07 0.0981751732473769
20 444 0.0007793846867637917 1.9496049720668772e-06
20 449 0.001550312014442292 0.09817703904473005
20 453 1.1861613167885076e-07 0.0981751732473769
20 471 0.0015502377489189495 0.09817157033779747
20 473 1.1861613167885076e-07 0.0981751732473769
20 474 0.0015505705266959378 0.09817681026601079
20 489 0.0007784571495218485 1.9555183931684476e-06
20 493 0.0015502706126583313 0.09817136924936228
20 504 0.0015503039206750382 0.0981726799203412
21 26 7.151206014641625e-08 0.09817440089467354
21 73 8.306690937511528e-08 0.09817359293409755
21 195 0.01589307724343903 0.03613590451612909
21 200 0.02514189261396959 0.09246144155881621
21 210 0.0016897343506939813 2.4673461316225318e-06
21 218 0.0020824534281974907 0.09817135145045462
21 229 0.0016887763049974074 3.601891472070439e-06
21 244 0.0012802017810659894 0.09816741348808125
21 247 0.002081685438323692 0.09817551559499965
21 252 0.001281299

27 281 0.00614378018623022 3.142838240427468e-06
27 364 0.005620168001952163 0.09818274506154115
27 368 0.006608355759795243 0.09817516098640522
27 374 0.006607222717387417 0.09816190732313582
27 391 0.006607942509245116 0.0981826583631159
27 397 0.006143619279557219 1.2707117560702131e-06
27 401 0.005619665026419186 0.098181981092888
27 428 0.006143652787997632 1.1991500579554984e-06
27 431 0.006608255363303533 0.0981747960129588
27 433 0.006143722708501979 4.2929829033875595e-07
27 451 0.006608637894965138 0.09817151260407067
27 458 0.006142975264961112 1.7827917643117326e-06
27 481 0.0061434331587475926 2.0726010052301897e-06
27 501 0.005619578716262489 0.09817492626985724
28 53 0.015296208730110224 0.020842170222279924
28 67 0.03865618769137331 0.10844732057041565
28 92 0.02473345651276815 0.0656859006827902
29 110 0.019232441565027566 0.06459856854571698
30 135 0.021333280716101844 0.13174700364461017
30 136 0.02102693567762272 0.11723898229647203
30 162 0.02692089103959544 0.0870

46 175 0.0031218512445573238 1.6281862725746506e-06
46 180 0.015183315601051756 0.09921577816478032
46 208 0.0031216350308200308 3.5964624738910964e-06
46 216 0.0028009427549512736 0.09817022449023223
46 223 0.0031216350308200546 3.5964624738910964e-06
46 230 0.0031216350308200546 3.5965242130863955e-06
46 239 0.0034134546293318304 0.09817568816663816
46 245 0.0028008956160761682 0.09817162222684202
46 260 0.002801301651255128 0.09817310383405574
46 286 0.005041881626589102 0.09817049193471346
46 290 0.006143610045325917 0.09817547704492093
46 307 0.0056193474940663644 3.5964624738910964e-06
46 322 0.005619347494066376 3.5965242130863955e-06
46 357 0.019102190240728702 0.1250984570868544
46 358 0.021158819227511925 0.11647151582404253
46 381 0.006143388154593131 0.09817725211624656
46 387 0.005620354524979558 1.4154950208010701e-05
46 421 0.0056193474940663644 3.5964624738910964e-06
46 425 0.005040321099756782 0.09818806004692109
46 447 0.005041033128891955 0.09817690847574115
46 485 0

63 138 7.934144352314445e-08 0.09817686477892977
63 177 0.0004329763256547921 5.176947345149637e-07
63 183 0.0008612507839015752 0.09817270917907756
63 184 0.0004333515885273595 1.9672354763066923e-06
63 192 0.01754839070375067 0.10082670510852454
63 207 0.0004324581870588144 3.2404115976691903e-06
63 211 0.0008608321372503796 0.09817115059779762
63 214 0.0008617900273479831 0.09817115059779762
63 217 0.000861197359192928 0.09817372626774976
63 225 7.934144352314445e-08 0.09817686477892977
63 226 0.0008612111176975242 0.09817225074880107
63 228 7.934144352314445e-08 0.09817686477892977
63 240 7.934144352314445e-08 0.09817686477892977
63 248 0.0004325238056593444 3.94669849817682e-06
63 263 1.3335566584146683e-06 0.09816748162731809
63 267 6.158320159713697e-07 0.09817686477892977
63 277 0.0015498650814719623 0.09816769082114926
63 293 6.158320159743494e-07 0.09817686477892977
63 334 0.0007792115281264746 1.1551043377092225e-06
63 366 0.0007794976166347724 7.912446437516576e-06
63 383 7

...

308 419 1.208571058873866e-06 1.6727547374628243e-05
308 435 0.0007787826121306557 0.0981526351772194
308 440 0.0007788341387788867 0.0981526351772194
308 460 1.6073922495343893e-06 2.455027696955653e-05
308 462 7.54990284182745e-07 3.1536884580365165e-05
308 468 1.4565857879724884e-06 1.7236379980064932e-05
308 483 0.000779533811471591 0.09821867933696345
308 508 0.0007787826121306557 0.0981526351772194
309 343 0.0007787056948283142 0.09816720329681698
309 353 0.026216105295435912 0.07690999463666287
309 356 0.02528564464189794 0.1179305236219771
309 359 0.0007793351359296151 0.09818107200380286
309 376 3.2847650787748407e-07 6.183640775129308e-06
309 386 0.0007789046603536776 0.09816906852673576
309 405 0.020915141596627416 0.09260129630273663
309 410 0.0007787155973644376 0.09816180569401432
309 411 3.254726442291317e-07 8.075700771913872e-06
309 413 2.784340817359033e-07 1.757872245290727e-05
309 420 0.0007785968138624418 0.09817011908027903
309 437 1.125132278206203e-06 3.78427890

364 397 0.0007788191746319691 0.09818147439367178
364 401 5.029938644555456e-07 7.641840818499564e-07
364 427 0.0007789023967004334 0.09816441397817059
364 428 0.0007788535933214805 0.09818394417014999
364 433 0.0007788959440610321 0.09818231579499412
364 455 0.0007788493528882935 0.09816514395265473
364 458 0.0007783618528298484 0.09818096227231297
364 478 0.0007791395399520094 0.09816366617104864
364 481 0.0007786841392640905 0.09818067254230871
364 494 0.0007795186135265877 0.09816454238838965
364 495 0.0007794186498206125 0.09817487349846503
364 501 5.909216743927198e-07 7.818808394984552e-06
364 507 0.0007791788173842301 0.09816613540620897
365 382 0.0007797075159262723 0.09818711682553587
365 416 0.0007794372837559891 0.0981776603379345
365 417 7.761696038180487e-07 4.118277675607573e-06
365 422 0.0007791802876243233 0.09819117650650885
365 426 0.0007787601714171449 0.09817469939379035
365 443 4.491924690106562e-07 4.087673979812578e-06
365 448 5.84938009395631e-07 1.188838136511

384 396 0.0007786175441486045 0.09816757371012992
384 424 0.0007792377392563728 0.09817596622085534
384 430 8.294136290860307e-08 4.243875822253306e-06
384 445 0.0007796025234545141 0.09816758871018197
384 450 2.727696719877022e-07 5.932459007917248e-07
384 454 0.0007790238233629822 0.09816979718531228
384 457 1.3427309661021528e-06 1.390561799901212e-05
384 475 0.0007789388350295571 0.09816946466395221
384 477 2.54526103632543e-07 5.389196074016495e-07
384 488 0.0007785275301579092 0.09817853028939182
384 490 0.0007787765340282709 0.09818317884305254
384 506 0.0007792695959181952 0.09817519570766685
385 390 0.0007799694637779591 0.09815036761509838
385 396 1.2431699350054031e-06 1.243829300168414e-05
385 427 0.0007786830324034476 0.09819728724761209
385 430 0.0007797636026660056 0.09815937930142538
385 445 1.3674395856121367e-07 1.2453288311792725e-05
385 450 0.0007794614344670326 0.09815572902737148
385 455 0.0007787464032127901 0.09819655727312816
385 457 0.0007808323255625311 0.098

407 482 0.0166918984577292 0.11383931667191356
407 486 0.01669478378416699 0.1138117400254756
407 492 0.01638564735148886 0.06533530973735098
408 432 8.17807329922207e-08 3.4850245049642988e-06
408 456 9.340784843828646e-07 4.092614162529039e-06
408 459 3.9742783281825516e-07 6.634079970360775e-06
408 467 7.6709729714267e-07 4.5197950995977135e-06
408 479 0.0007788088007561401 0.0981745316954935
408 482 8.23874712618871e-07 1.559111564825918e-05
408 486 2.172343315494359e-06 4.924809653609687e-05
408 492 0.0007789395021755384 0.09819019268311881
408 505 0.0007789264441096963 0.09816666386461272
408 509 0.0007785574049580847 0.09816073676917303
409 434 0.0007790017500523175 0.09817616364847943
409 435 0.000778913212990795 0.09816941124441565
409 439 0.0007790371123929291 0.09817926516189375
409 440 0.0007789657461153339 0.09816941124441565
409 461 0.0007789898121918691 0.09817551917900087
409 508 0.000778913212990795 0.09816941124441565
410 411 0.0007788904462674191 0.09816988140099063


438 465 2.2588510519271306e-06 3.612837466890411e-05
438 491 0.0007806892063779838 0.09813797655695043
438 502 0.0007773082063967659 0.09819551513981352
439 461 1.8841235211281067e-07 3.745916011291342e-06
439 492 0.0007790476484722669 0.09816523759095512
440 460 0.000779027268346857 0.09817718545608095
440 462 0.0007790827266661862 0.09818417206097878
440 468 0.0007789693838758404 0.09816987156156252
440 508 5.368235319779949e-07 0.0
441 446 0.0007795666665480411 0.09818768489461724
441 463 0.0007784964519324599 0.09817436526661157
441 464 1.9800339878600296e-06 2.163527504408852e-05
441 472 2.811959864050182e-07 6.658851943259308e-06
441 484 0.0007786289256287039 0.09817272041364021
441 510 0.0007790624409309579 0.09817904579763612
442 447 0.000778338189229126 0.09816955882362606
442 452 0.000780116456326486 0.098188497252822
442 465 3.7807254823977266e-07 1.0789037557604385e-05
442 491 0.0007793068348817041 0.09816331589872214
442 502 0.0007789111935238203 0.09817017579804548
443 44

In [194]:
k1, k2

(array([ 0.08726476,  0.0731308 ,  0.0400593 , ..., -2.54055871,
        -2.54062088, -2.54056058]),
 array([5.67214749e+00, 6.04994646e+00, 5.55570293e+00, ...,
        1.41687329e-03, 1.41637564e-03, 1.07589602e-04]))

In [281]:
lxcPairs[22], lxcPairs[250], lxcPairs[271], lxcPairs[501]

([259,
  285,
  array([ 0.70710678,  0.70710678, -0.        ]),
  array([0., 0., 0.])],
 [550,
  570,
  array([ 0.70710678,  0.70710678, -0.        ]),
  array([-0.02056, -0.02056,  0.     ])],
 [587,
  597,
  array([ 0.70710678,  0.70710678, -0.        ]),
  array([0., 0., 0.])],
 [996,
  1020,
  array([ 0.70710678,  0.70710678, -0.        ]),
  array([-0.03701, -0.03701,  0.     ])])

In [208]:
v[lxcPairs[22][0]], v[lxcPairs[22][1]], v[lxcPairs[271][0]], v[lxcPairs[271][1]]

(array([ 0.19729, -0.99185,  6.80349]),
 array([ 0.99185, -0.19729,  6.80349]),
 array([-1.25006, -0.24865,  5.08451]),
 array([0.24865, 1.25006, 5.08451]))

In [299]:
x = np.array([7, 8])
y = np.array([6, 10])

c = x > y
print(any(c), all(c))

True False


In [301]:
cx = distanceMetricRx(lxcPairs[22], lxcPairs[250]) > y
print(any(c), all(c))

True False


In [305]:
p3 = []
for i in range(0, len(p3)):
    print(i)

In [422]:
set_clus = []
clus = [[0.0, 2.0, 4.0], [], []]
set_clus.append(clus)
set_clus, clus[1], clus[2]

([[[0.0, 2.0, 4.0], [], []]], [], [])

In [315]:
clus2 = [[0.0, 2.0, 4.0], np.array([]), np.array([])]

In [320]:
clus[1].append(lxcPairs[0])
clus[2].append(shiftedPairsFx[0])

In [321]:
clus

[[0.0, 2.0, 4.0],
 [[4,
   33,
   array([ 0.14220045, -0.65268028,  0.74416899]),
   array([ 0.56531429, -2.59471394,  2.95842496])]],
 [[4,
   33,
   array([ 0.14220045, -0.65268028,  0.74416899]),
   array([ 0.56531429, -2.59471394,  2.95842496])]]]